In [20]:
import tensorflow as tf
import os
import os.path

class main(object):
    
    PATH_DATASET = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset'
    MFCC_DIM = 39
    BATCH_SIZE = 30
    LEN_SEQ = 300
    
    def input(self, path_tfr, MFCC_DIM, BATCH_SIZE):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)
        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })
        
        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)
        
        datas = tf.reshape(datas, [self.LEN_SEQ, MFCC_DIM])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.shuffle_batch(
            [datas, labels],
            batch_size=BATCH_SIZE,capacity=1000+MFCC_DIM*BATCH_SIZE,min_after_dequeue=1000
        )
            
        return datas, labels
    
    def train(self):
        x = tf.placeholder(tf.float32, shape=[None, self.LEN_SEQ, self.MFCC_DIM])

        datas, labels = self.input(os.path.join(self.PATH_DATASET, 'train-male.tfrecords'), self.MFCC_DIM, self.BATCH_SIZE)
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        
        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            batch = sess.run([datas, labels])
            print(batch[1])
            coord.request_stop()
            coord.join(threads)

In [21]:
m = main()
#m.input(os.path.join(m.PATH_DATASET, 'train-male.tfrecords'), m.MFCC_DIM, m.BATCH_SIZE)
m.train()

[[ 0]
 [11]
 [ 5]
 [ 8]
 [10]
 [10]
 [15]
 [ 4]
 [18]
 [ 7]
 [16]
 [21]
 [ 8]
 [ 7]
 [11]
 [ 8]
 [21]
 [ 6]
 [12]
 [ 2]
 [14]
 [21]
 [18]
 [ 6]
 [ 5]
 [18]
 [21]
 [ 6]
 [ 0]
 [ 8]]
